In [1]:
import pandas as pd
import textstat
import os

In [2]:
def readability(essay):
    return textstat.flesch_reading_ease(essay)

def ttr(essay):
    words = essay.split()
    unique_words = set(words)
    return len(unique_words) / len(words)

In [3]:
path = os.path.dirname(os.getcwd())
test_set = pd.read_csv(os.path.join(path, 'data', 'joint_test_dataset.csv'))
test_set['Readability'] = test_set['EssayText'].apply(readability)
test_set['TTR'] = test_set['EssayText'].apply(ttr)
test_set

,Id,EssaySet,EssayText,EssayWeights,Score1,Readability,TTR
0,1673,1,The procedures I think they should have includ...,1,1,71.14,0.823529
1,1674,1,"In order to replicate this experiment, you wou...",1,1,60.32,0.805556
2,1675,1,"In order to replicate their experiment, you wo...",1,3,64.04,0.697917
3,1676,1,Pleace a simple of one material into one conta...,1,0,42.38,0.791667
4,1677,1,Determin the mass of four different samples ma...,1,0,84.68,0.777778
...,...,...,...,...,...,...,...
5219,28130,10,light gray :: The tempature of the dog house w...,1,1,88.06,0.800000
5220,28131,10,light gray :: I choose dark gray because the l...,1,1,92.83,0.787879
5221,28132,10,dark gray :: Dark gray is better because it wi...,1,1,86.03,0.950000
5222,28133,10,white :: the heat would be less atracted to wh...,1,1,98.55,0.812500


In [4]:
questions = {1: 'A group of students wrote the following procedure for their investigation. Procedure:\
- Determine the mass of four different samples.\
- Pour vinegar in each of four separate, but identical, containers.\
- Place a sample of one material into one container and label. Repeat with remaining samples, placing a single sample into a single container.\
- After 24 hours, remove the samples from the containers and rinse each sample with distilled water.\
- Allow the samples to sit and dry for 30 minutes. \
- Determine the mass of each sample.\
The data recorded by the students are recorded in a table containing information about wood, plastic, marble and limestone and their respective starting mass, ending mass and difference in mass.\
After reading the group’s procedure, describe what additional information you would need in order to replicate the experiment. Make sure to include at least three pieces of information.',

2 :'A student performed the following investigation to test four different polymer plastics for stretchability.\
Procedure:\
- Take a sample of one type of plastic, and measure its length.  \
- Tape the top edge of the plastic sample to a table so that it is hanging freely down the side of the table.\
- Attach a clamp to the bottom edge of the plastic sample.\
- Add weights to the clamp and allow them to hang for five minutes.\
- Remove the weights and clamp, and measure the length of the plastic types.\
- Repeat the procedure exactly for the remaining three plastic samples.\
- Perform a second trial (T2) exactly like the first trial (T1).\
The student recorded the data from the investigation in atable containing the stretches over the trials of the different kind of plastics.',

3 : 'Explain how pandas in China are similar to koalas in Australia and how they both are different from pythons. Support your response with information from the article.',

4 :'Explain the significance of the word “invasive” to the rest of the article. Support your response with information from the article.', 
5 :'Starting with mRNA leaving the nucleus, list and describe four major steps involved in protein synthesis.',
6 :'List and describe three processes used by cells to control the movement of substances across the cell membrane.',
7 :'Identify ONE trait that can describe Rose based on her conversations with Anna or Aunt Kolab. Include ONE detail from the story that supports your answer.',
8 :'During the story, the reader gets background information about Mr. Leonard. Explain the effect that background information has on Paul. Support your response with details from the story.',
9 :'How does the author organize the article? Support your response with details from the article.',
10 :'Brandi and Jerry did the following controlled experiment to find out how the color of an object affects its temperature.\
Question: What is the effect of different lid colors on the air temperature inside a glass jar exposed to a lamp?\
Hypothesis: The darker the lid color, the greater the increase in air temperature in the glass jar, because darker colors absorb more energy.\
Materials:\
glass jar\
lamp\
four colored lids: black, dark gray, light gray, and white\
thermometer\
meterstick\
stopwatch\
Procedure:\
Put the black lid with the attached thermometer on the glass jar.\
Make sure the starting temperature inside the jar is 24° C.\
Place lamp 5 centimeters away from the lid and turn on the lamp.\
After 10 minutes measure the air temperature inside the glass jar and record as Trial 1.\
Turn off lamp and wait until the air in the jar returns to the starting temperature.\
Repeat steps 2 through 5 for Trials 2 and 3.\
Repeat steps 1 through 6 for the dark gray, light gray, and white lids.\
Calculate and record the average air temperature for each lid color.\
Brandi and Jerry were designing a doghouse. Use the results from the experiment to describe the best paint color for the doghouse.\
In your description, be sure to:\
Choose a paint color. \
Describe how that color might affect the inside of the doghouse.\
Use results from the experiment to support your description.'
}

: 

In [ ]:
import transformers
from transformers import BertTokenizer
from src.prompt_creation import CreatePrompt
import os
import torch 
import pandas as pd

/Users/juliabelloni/anaconda3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
text_model = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline("text-generation", model=text_model, tokenizer=tokenizer  model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")

path = os.path.join(os.getcwd(), 'data', 'test.csv')
get_prompt = CreatePrompt()
df_test =pd.read_csv(path)

pos = 0
final_score = df_test['Score1'].iloc[pos]
essay_set = df_test['EssaySet'].iloc[pos]
readability = df_test['Readability'].iloc[pos]
ttr = df_test['Readability'].iloc[pos]
answer = df_test['EssayText'].iloc[pos]
prompt = get_prompt.get_prompt(essay_set,final_score,readability, ttr, answer)

pipeline(prompt)